In [1]:
import cropvalidation 
import os
import numpy as np
import pandas as pd

# Togo Data


Load in togo data as cropmasks

In [2]:
togo_data = []
dir = 'data/TogoBinaryMaps'
for tif in os.listdir(dir):
    f = os.path.join(dir, tif)
    if f == "data/TogoBinaryMaps\AgreementMap_Togo_Binary.tif":
        continue
    togo_data.append(cropvalidation.CropMask(f))

togo_labels = [togo_data[x].getName().split("\\")[-1][:-11] for x in np.arange(len(togo_data))]

TOGO_BINARY = dict(zip(togo_labels, togo_data))


Read ground-truth csv as validation data


In [ ]:
#create validation object
TOGO_VALIDATION = cropvalidation.ValidationSet(path = 'data/HarvestData/togo.csv', projection='epsg:4326')

#data preprocessing
TOGO_VALIDATION.validation = TOGO_VALIDATION.validation[TOGO_VALIDATION.validation['subset'] == 'testing']
#convert agreement values to discrete labels
TOGO_VALIDATION.validation.loc[TOGO_VALIDATION.validation['crop_probability'].astype(float) >= 0.5, 'crop_probability'] = 1
TOGO_VALIDATION.validation.loc[TOGO_VALIDATION.validation['crop_probability'].astype(float) < 0.5, 'crop_probability'] = 0
TOGO_VALIDATION.validation = TOGO_VALIDATION.validation[['lat', 'lon', 'crop_probability']]

#removing non-data values
TOGO_VALIDATION.filter()
TOGO_VALIDATION.reproject_validation(outProjection='epsg:32631')


In [ ]:
TOGO_VALIDATION.getData()

,lat,lon,crop_probability
77,170560.155817,1.188884e+06,1
116,172692.923816,1.193288e+06,0
138,175396.863868,1.175290e+06,1
140,177454.027895,1.221734e+06,1
145,181613.837353,1.171039e+06,1
...,...,...,...
1537,346193.954748,8.081821e+05,1
1544,346361.833763,7.019713e+05,1
1561,348127.594121,9.458717e+05,0
1605,349511.948410,7.003744e+05,1


Create matched datasets

In [ ]:
TOGO_MATCHED = []
for binarymap in np.arange(len(togo_data)):
    TOGO_MATCHED.append(cropvalidation.MatchedSet(togo_data[binarymap], TOGO_VALIDATION))

In [ ]:
TOGO_MATCHED[0].data

,lat,lon,crop_probability,cropmask
0,170560.155817,1.188884e+06,1.0,1.0
1,172692.923816,1.193288e+06,0.0,1.0
2,175396.863868,1.175290e+06,1.0,0.0
3,177454.027895,1.221734e+06,1.0,1.0
4,181613.837353,1.171039e+06,1.0,1.0
...,...,...,...,...
350,346193.954748,8.081821e+05,1.0,1.0
351,346361.833763,7.019713e+05,1.0,1.0
352,348127.594121,9.458717e+05,0.0,0.0
353,349511.948410,7.003744e+05,1.0,1.0


Create classification reports

In [ ]:
TOGO_REPORT = pd.DataFrame(columns=['F1 Score', 'Total Accuracy', 'Crop Precision', 'Non-Crop Precision', 'Crop Recall', 'Non-Crop Recall', 'Points'], index = [x.split('/')[-1] for x in togo_labels])

#create reports showing f1, total accuracy, precision, recall, and number of datapoints processed for each matched set. 
for binarymap in np.arange(len(TOGO_MATCHED)):
    TOGO_MATCHED[binarymap].remove_filler()
    TOGO_REPORT.iloc[binarymap] = TOGO_MATCHED[binarymap].report(show_size=True)
    

**Togo classification report**

In [ ]:
TOGO_REPORT.sort_values(by = 'Total Accuracy', ascending=False)

,F1 Score,Total Accuracy,Crop Precision,Non-Crop Precision,Crop Recall,Non-Crop Recall,Points
Harvest_Togo,0.73125,0.757746,0.696429,0.812834,0.748768,0.769737,355.0
ESA_Togo,0.618026,0.749296,0.888889,0.708029,0.955665,0.473684,355.0
DigitalEarth_Togo,0.560669,0.704225,0.770115,0.682836,0.901478,0.440789,355.0
CombinedMap_Togo,0.502203,0.68169,0.76,0.660714,0.91133,0.375,355.0
GFSAD_Togo,0.580882,0.678873,0.658333,0.689362,0.79803,0.519737,355.0
ASAP_Togo,0.471074,0.639437,0.633333,0.641509,0.837438,0.375,355.0
ESRI_Togo,0.109091,0.585915,0.692308,0.581871,0.980296,0.059211,355.0
Copernicus_Togo,0.192513,0.574648,0.514286,0.58125,0.916256,0.118421,355.0


# Kenya Data

In [1]:
import cropvalidation 
import os
import numpy as np
import pandas as pd

In [2]:
kenya_data = []
dir = 'data/KenyaBinaryMaps'
for tif in os.listdir(dir):
    f = os.path.join(dir, tif)
    if f == "data/KenyaBinaryMaps/Agreement_Kenya_Binary.tif":
        continue
    kenya_data.append(cropvalidation.CropMask(f))

kenya_labels = [kenya_data[x].getName().split("\\")[-1][:-11] for x in np.arange(len(kenya_data))]

KENYA_BINARY = dict(zip(kenya_labels, kenya_data))
kenya_data[0].getProjection()

CRS.from_epsg(32736)

In [3]:
KENYA_VALIDATION = cropvalidation.ValidationSet(path = 'data/HarvestData/kenya.csv', projection='epsg:4326')
KENYA_VALIDATION.validation = KENYA_VALIDATION.validation[KENYA_VALIDATION.validation['subset'] == 'testing']
KENYA_VALIDATION.validation = KENYA_VALIDATION.validation[['lat', 'lon', 'crop_probability']]

KENYA_VALIDATION.validation.crop_probability = KENYA_VALIDATION.validation.crop_probability.astype(float)

KENYA_VALIDATION.filter()
KENYA_VALIDATION.reproject_validation(outProjection='epsg:32736')

In [4]:
KENYA_MATCHED = []
for binarymap in np.arange(len(kenya_data)):
    KENYA_MATCHED.append(cropvalidation.MatchedSet(kenya_data[binarymap], KENYA_VALIDATION))

In [ ]:
kenya_labels = [kenya_data[x].getName().split("\\")[-1][:-11] for x in np.arange(len(kenya_data))]
KENYA_REPORT = pd.DataFrame(columns=['F1 Score', 'Total Accuracy', 'Crop Precision', 'Non-Crop Precision', 'Crop Recall', 'Non-Crop Recall', 'Points'], index = [x.split('/')[-1] for x in kenya_labels])

for binarymap in np.arange(len(KENYA_MATCHED)):
    KENYA_MATCHED[binarymap].remove_filler()
    KENYA_REPORT.iloc[binarymap] = KENYA_MATCHED[binarymap].report(show_size=True)


**Kenya classification report**

In [ ]:
KENYA_REPORT.sort_values(by = 'Total Accuracy', ascending=False)

,F1 Score,Total Accuracy,Crop Precision,Non-Crop Precision,Crop Recall,Non-Crop Recall,Points
GFSAD_Kenya,0.973451,0.959108,0.951258,0.988304,0.845,0.996705,807.0
Harvest_Kenya,0.967309,0.951673,0.984642,0.864253,0.955,0.950577,807.0
CombinedMap_Kenya,0.965633,0.949195,0.982935,0.859729,0.95,0.948929,807.0
Asap_Kenya,0.963739,0.944238,0.943218,0.947977,0.82,0.985173,807.0
DE_Kenya,0.932575,0.89715,0.919872,0.819672,0.75,0.945634,807.0
Agreement_Kenya,0.068966,0.834356,0.035714,1.0,0.833333,1.0,163.0
ESRI_Kenya,0.707724,0.653036,0.965812,0.412281,0.94,0.558484,807.0
EsaWorldcover_Kenya,0.305132,0.379182,0.964912,0.282828,0.98,0.181219,807.0
Copernicus_Kenya,0.190193,0.324659,0.969697,0.267206,0.99,0.105437,807.0
